### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
school_data_to_load = "schools_complete.csv"
student_data_to_load = "students_complete.csv"

# Read School and Student Data File and store into Pandas Data Frames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)
school_data['Per Student Budget'] = school_data['budget']/school_data['size']





In [2]:
Total_Students = student_data["Student ID"].count()
Total_Schools = school_data["School ID"].count()
Total_Budget = school_data["budget"].sum()
Avg_Math = student_data["math_score"].mean()
Avg_Reading = student_data["reading_score"].mean()
Math_O_70 = student_data.loc[student_data["math_score"]>69,"Student ID"].count()
Math_Pct = (Math_O_70/Total_Students)*100
Read_O_70 = student_data.loc[student_data["reading_score"]>69,"Student ID"].count()
Read_Pct = (Read_O_70/Total_Students)*100
Overall = (Math_Pct+Read_Pct)/2


summary_df = pd.DataFrame({"Total Students": [Total_Students],
                           "Total Schools": [Total_Schools],
                           "Total Budget": [Total_Budget],
                           "Avg Math": [Avg_Math],
                           "Avg Reading": [Avg_Reading],
                           "Math % >= 70":[Math_Pct],
                          "Read % >= 70":[Read_Pct],
                           "Overall": [Overall],
                          })
summary_df    



,Total Students,Total Schools,Total Budget,Avg Math,Avg Reading,Math % >= 70,Read % >= 70,Overall
0,39170,15,24649428,78.985371,81.87784,74.980853,85.805463,80.393158


In [3]:
school_data_new = school_data
by_school = student_data.groupby(['school_name'])
school_avg_df= by_school.mean()
school_avg_df = school_avg_df.rename(columns={'reading_score':'Avg Reading','math_score':'Avg Math'})
del school_avg_df['Student ID']

school_data_new = school_data_new.merge(school_avg_df, on='school_name', how="outer")

stu_data_tmp = student_data[['school_name', 'reading_score', 'math_score']].copy()
cnt_passing_read = stu_data_tmp[stu_data_tmp['reading_score']>=70]
del cnt_passing_read['math_score']
cnt_passing_math = stu_data_tmp[stu_data_tmp['math_score']>=70]
del cnt_passing_math['reading_score']

reading_cnt = cnt_passing_read.groupby(['school_name']).count()
math_cnt = cnt_passing_math.groupby(['school_name']).count()


cnt_pass = math_cnt.merge(reading_cnt, on='school_name', how="inner")

student_data_new = school_data_new.merge(cnt_pass, on='school_name', how="outer")

student_data_new['Reading Pass Rate'] = student_data_new['reading_score']/student_data_new['size']*100
student_data_new['Math Pass Rate'] = student_data_new['math_score']/student_data_new['size']*100
student_data_new['Overall Rate'] = (student_data_new['Reading Pass Rate']+student_data_new['Math Pass Rate'])/2
del student_data_new['reading_score']
del student_data_new['math_score']

student_data_new

,School ID,school_name,type,size,budget,Per Student Budget,Avg Reading,Avg Math,Reading Pass Rate,Math Pass Rate,Overall Rate
0,0,Huang High School,District,2917,1910635,655.0,81.182722,76.629414,81.316421,65.683922,73.500171
1,1,Figueroa High School,District,2949,1884411,639.0,81.158020,76.711767,80.739234,65.988471,73.363852
2,2,Shelton High School,Charter,1761,1056600,600.0,83.725724,83.359455,95.854628,93.867121,94.860875
3,3,Hernandez High School,District,4635,3022020,652.0,80.934412,77.289752,80.862999,66.752967,73.807983
4,4,Griffin High School,Charter,1468,917500,625.0,83.816757,83.351499,97.138965,93.392371,95.265668
5,5,Wilson High School,Charter,2283,1319574,578.0,83.989488,83.274201,96.539641,93.867718,95.203679
6,6,Cabrera High School,Charter,1858,1081356,582.0,83.975780,83.061895,97.039828,94.133477,95.586652
7,7,Bailey High School,District,4976,3124928,628.0,81.033963,77.048432,81.933280,66.680064,74.306672
8,8,Holden High School,Charter,427,248087,581.0,83.814988,83.803279,96.252927,92.505855,94.379391
9,9,Pena High School,Charter,962,585858,609.0,84.044699,83.839917,95.945946,94.594595,95.270270


## Top Performing Schools (By Passing Rate)

In [4]:
top_performing = student_data_new.sort_values(by=['Overall Rate'], ascending=False).head(5)
top_performing

,School ID,school_name,type,size,budget,Per Student Budget,Avg Reading,Avg Math,Reading Pass Rate,Math Pass Rate,Overall Rate
6,6,Cabrera High School,Charter,1858,1081356,582.0,83.975780,83.061895,97.039828,94.133477,95.586652
14,14,Thomas High School,Charter,1635,1043130,638.0,83.848930,83.418349,97.308869,93.272171,95.290520
9,9,Pena High School,Charter,962,585858,609.0,84.044699,83.839917,95.945946,94.594595,95.270270
4,4,Griffin High School,Charter,1468,917500,625.0,83.816757,83.351499,97.138965,93.392371,95.265668
5,5,Wilson High School,Charter,2283,1319574,578.0,83.989488,83.274201,96.539641,93.867718,95.203679


* Sort and display the top five schools in overall passing rate

## Bottom Performing Schools (By Passing Rate)

In [5]:
bottom_performing = student_data_new.sort_values(by=['Overall Rate'], ascending=True).head(5)
bottom_performing

,School ID,school_name,type,size,budget,Per Student Budget,Avg Reading,Avg Math,Reading Pass Rate,Math Pass Rate,Overall Rate
11,11,Rodriguez High School,District,3999,2547363,637.0,80.744686,76.842711,80.220055,66.366592,73.293323
1,1,Figueroa High School,District,2949,1884411,639.0,81.158020,76.711767,80.739234,65.988471,73.363852
0,0,Huang High School,District,2917,1910635,655.0,81.182722,76.629414,81.316421,65.683922,73.500171
12,12,Johnson High School,District,4761,3094650,650.0,80.966394,77.072464,81.222432,66.057551,73.639992
13,13,Ford High School,District,2739,1763916,644.0,80.746258,77.102592,79.299014,68.309602,73.804308


## Math Scores by Grade

In [6]:
G9 = student_data.loc[student_data["grade"] == "9th"].groupby("school_name")["math_score"].mean()
G10 = student_data.loc[student_data["grade"] == "10th"].groupby("school_name")["math_score"].mean()
G11 = student_data.loc[student_data["grade"] == "11th"].groupby("school_name")["math_score"].mean()
G12 = student_data.loc[student_data["grade"] == "12th"].groupby("school_name")["math_score"].mean()

math_scores = pd.DataFrame({
        "9th": G9,
        "10th": G10,
        "11th": G11,
        "12th": G12
})

math_scores = math_scores[["9th", "10th", "11th", "12th"]]
math_scores.index.name = " "
math_scores

,9th,10th,11th,12th
,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [8]:
G9 = student_data.loc[student_data["grade"] == "9th"].groupby("school_name")["reading_score"].mean()
G10 = student_data.loc[student_data["grade"] == "10th"].groupby("school_name")["reading_score"].mean()
G11 = student_data.loc[student_data["grade"] == "11th"].groupby("school_name")["reading_score"].mean()
G12 = student_data.loc[student_data["grade"] == "12th"].groupby("school_name")["reading_score"].mean()

reading_scores = pd.DataFrame({
        "9th": G9,
        "10th": G10,
        "11th": G11,
        "12th": G12
})

reading_scores = reading_scores[["9th", "10th", "11th", "12th"]]
reading_scores.index.name = " "
reading_scores

,9th,10th,11th,12th
,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


## Scores by School Spending

In [16]:
spend_bins = [0, 585, 605, 630, 655]
spend_labels = ["Less than $585", "Medium $585-615", "Average $615-645", "Above Average $645-675"]
school_data_bud = school_data
school_data_bud["Spending Range"] = pd.cut(school_data_bud["Per Student Budget"], spend_bins, labels=spend_labels)

student_data_new_bud = school_data_bud.merge(student_data, on='school_name', how="inner")
by_school_bud = student_data_new_bud.groupby(['Spending Range'])
school_avg_df_bud = by_school_bud.mean()
school_avg_df_bud = school_avg_df_bud.rename(columns={'reading_score':'Avg Reading','math_score':'Avg Math'})


stu_data_tmp = student_data_new_bud[['Spending Range', 'reading_score', 'math_score']].copy()
cnt_passing_read = stu_data_tmp[stu_data_tmp['reading_score']>=70]
del cnt_passing_read['math_score']
cnt_passing_math = stu_data_tmp[stu_data_tmp['math_score']>=70]
del cnt_passing_math['reading_score']

all_cnt = stu_data_tmp.groupby(['Spending Range'])
all_cnt1 = all_cnt.count()

reading_cnt = cnt_passing_read.groupby(['Spending Range']).count()
math_cnt = cnt_passing_math.groupby(['Spending Range']).count()


cnt_pass = math_cnt.merge(reading_cnt, on='Spending Range', how="inner")
cnt_pass1 = cnt_pass.merge(all_cnt1, on='Spending Range', how="inner")
cnt_pass1 = cnt_pass1.rename(columns={'reading_score_y':'all_scores'})

student_data_new_bud = school_avg_df_bud.merge(cnt_pass1, on='Spending Range', how="outer")

student_data_new_bud['Reading Pass Rate'] = student_data_new_bud['reading_score_x']/student_data_new_bud['all_scores']*100
student_data_new_bud['Math Pass Rate'] = student_data_new_bud['math_score_x']/student_data_new_bud['all_scores']*100
student_data_new_bud['Overall Rate'] = (student_data_new_bud['Reading Pass Rate']+student_data_new_bud['Math Pass Rate'])/2
del student_data_new_bud['Student ID']
del student_data_new_bud['School ID']
del student_data_new_bud['size']
del student_data_new_bud['math_score_x']
del student_data_new_bud['reading_score_x']
del student_data_new_bud['budget']
del student_data_new_bud['all_scores']
del student_data_new_bud['math_score_y']
del student_data_new_bud['Per Student Budget']

student_data_new_bud

,Avg Reading,Avg Math,Reading Pass Rate,Math Pass Rate,Overall Rate
Spending Range,,,,,
Less than $585,83.964039,83.363065,96.686558,93.702889,95.194724
Medium $585-615,83.725724,83.359455,95.854628,93.867121,94.860875
Average $615-645,81.976641,79.179989,86.767486,75.600864,81.184175
Above Average $645-675,81.147112,77.418997,81.823567,68.335096,75.079331


## Scores by School Size

* Perform the same operations as above, based on school size.

In [13]:
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]
school_data_siz = school_data
school_data_siz["School Size"] = pd.cut(school_data_siz["size"], size_bins, labels=group_names)

student_data_new_siz = school_data_siz.merge(student_data, on='school_name', how="inner")
by_school_siz = student_data_new_siz.groupby(['School Size'])
school_avg_df_siz = by_school_siz.mean()
school_avg_df_siz = school_avg_df_siz.rename(columns={'reading_score':'Avg Reading','math_score':'Avg Math'})


stu_data_tmp = student_data_new_siz[['School Size', 'reading_score', 'math_score']].copy()
cnt_passing_read = stu_data_tmp[stu_data_tmp['reading_score']>=70]
del cnt_passing_read['math_score']
cnt_passing_math = stu_data_tmp[stu_data_tmp['math_score']>=70]
del cnt_passing_math['reading_score']

all_cnt = stu_data_tmp.groupby(['School Size'])
all_cnt1 = all_cnt.count()

reading_cnt = cnt_passing_read.groupby(['School Size']).count()
math_cnt = cnt_passing_math.groupby(['School Size']).count()


cnt_pass = math_cnt.merge(reading_cnt, on='School Size', how="inner")
cnt_pass1 = cnt_pass.merge(all_cnt1, on='School Size', how="inner")
cnt_pass1 = cnt_pass1.rename(columns={'reading_score_y':'all_scores'})

student_data_new_siz = school_avg_df_siz.merge(cnt_pass1, on='School Size', how="outer")

student_data_new_siz['Reading Pass Rate'] = student_data_new_siz['reading_score_x']/student_data_new_siz['all_scores']*100
student_data_new_siz['Math Pass Rate'] = student_data_new_siz['math_score_x']/student_data_new_siz['all_scores']*100
student_data_new_siz['Overall Rate'] = (student_data_new_siz['Reading Pass Rate']+student_data_new_siz['Math Pass Rate'])/2
del student_data_new_siz['Student ID']
del student_data_new_siz['School ID']
del student_data_new_siz['size']
del student_data_new_siz['math_score_x']
del student_data_new_siz['reading_score_x']
del student_data_new_siz['budget']
del student_data_new_siz['all_scores']
del student_data_new_siz['math_score_y']
del student_data_new_siz['Per Student Budget']

student_data_new_siz

,Avg Reading,Avg Math,Reading Pass Rate,Math Pass Rate,Overall Rate
School Size,,,,,
Small (<1000),83.974082,83.828654,96.040317,93.952484,94.996400
Medium (1000-2000),83.867989,83.372682,96.773058,93.616522,95.194790
Large (2000-5000),81.198674,77.477597,82.125158,68.652380,75.388769


## Scores by School Type

* Perform the same operations as above, based on school type.

In [11]:
school_data_typ = school_data
school_data_typ = school_data_typ.rename(columns={'type':'School Type'})
student_data_new_typ = school_data_typ.merge(student_data, on='school_name', how="inner")
by_school_typ = student_data_new_typ.groupby(['School Type'])
school_avg_df_typ = by_school_typ.mean()
school_avg_df_typ = school_avg_df_typ.rename(columns={'reading_score':'Avg Reading','math_score':'Avg Math'})



stu_data_tmp = student_data_new_typ[['School Type', 'reading_score', 'math_score']].copy()
cnt_passing_read = stu_data_tmp[stu_data_tmp['reading_score']>=70]
del cnt_passing_read['math_score']
cnt_passing_math = stu_data_tmp[stu_data_tmp['math_score']>=70]
del cnt_passing_math['reading_score']

all_cnt = stu_data_tmp.groupby(['School Type'])
all_cnt1 = all_cnt.count()

reading_cnt = cnt_passing_read.groupby(['School Type']).count()
math_cnt = cnt_passing_math.groupby(['School Type']).count()


cnt_pass = math_cnt.merge(reading_cnt, on='School Type', how="inner")
cnt_pass1 = cnt_pass.merge(all_cnt1, on='School Type', how="inner")
cnt_pass1 = cnt_pass1.rename(columns={'reading_score_y':'all_scores'})

student_data_new_typ = school_avg_df_typ.merge(cnt_pass1, on='School Type', how="outer")

student_data_new_typ['Reading Pass Rate'] = student_data_new_typ['reading_score_x']/student_data_new_typ['all_scores']*100
student_data_new_typ['Math Pass Rate'] = student_data_new_typ['math_score_x']/student_data_new_typ['all_scores']*100
student_data_new_typ['Overall Rate'] = (student_data_new_typ['Reading Pass Rate']+student_data_new_typ['Math Pass Rate'])/2
del student_data_new_typ['Student ID']
del student_data_new_typ['School ID']
del student_data_new_typ['math_score_x']
del student_data_new_typ['reading_score_x']
del student_data_new_typ['budget']
del student_data_new_typ['all_scores']
del student_data_new_typ['math_score_y']
del student_data_new_typ['Per Student Budget']
del student_data_new_typ['size']


In [14]:
student_data_new_typ

,Avg Reading,Avg Math,Reading Pass Rate,Math Pass Rate,Overall Rate
School Type,,,,,
Charter,83.902821,83.406183,96.645891,93.701821,95.173856
District,80.962485,76.987026,80.905249,66.518387,73.711818


In [17]:
school_avg_df_typ

,School ID,size,budget,Per Student Budget,Student ID,Avg Reading,Avg Math
School Type,,,,,,,
Charter,6.964163,1717.352468,1.024543e+06,598.778498,19438.475398,83.902821,83.406183
District,6.984505,4063.261195,2.611175e+06,643.087300,19650.507711,80.962485,76.987026
